In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pydantic import BaseModel
class TestStructuredOutput(BaseModel):
    a: int
    b: str
    

In [ ]:
def get_weather(location: str):
    """Get current temperature for a given location."""
    raise ValueError("connection issue. Please try again later.")  # Simulating an error for demonstration
    return f"The current temperature in {location} is 72°F."

In [ ]:
import os
from neo.mcp.client import MCPClient

module_dir = os.path.dirname(__import__("neo").__file__)
example_dir = os.path.join(module_dir, "../../examples")
mcp_client = MCPClient(
    name="demo",
    server_script_path=f"{example_dir}/mcp/mcp-server-demo/main.py",
)

# Anthropic

In [ ]:
from neo.models.providers.anthropic import AnthropicModel

model = AnthropicModel(model='claude-sonnet-4-20250514',
                       enable_thinking=True,
                           boolean_response=False, 
                           structured_response_model=None,
                        #    tools=[get_weather],
                        #    mcp_clients=[mcp_client],
                           )
test = await model.acreate("what is the weather today in new york?", return_response_object=False)

In [ ]:
test.display()

In [ ]:
test.get_context(-1).contents[0]

In [ ]:
await model.acreate(user_input=None, base_thread=test)

In [ ]:
test.display()

# Image

In [ ]:
from neo.models.providers.anthropic import AnthropicModel

model = AnthropicModel(model='claude-3-7-sonnet-20250219')

In [ ]:
from neo.contexts import Context
from neo.types import contents as t

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

context = Context(
    contents=[
        t.TextContent(data="What's in this image?"),
        t.ImageContent(data=image_url),
    ]
)

In [ ]:
thread = await model.acreate(user_input=context)

In [ ]:
thread.display()

In [ ]:
await model.acreate("make a poem about this image", base_thread=thread)  
thread.display()

# Document

In [ ]:
txt_url = "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"

In [ ]:
from neo.models.providers.anthropic import AnthropicModel

model = AnthropicModel(model='claude-3-7-sonnet-20250219')

from neo.contexts import Context
from neo.types import contents as t


context = Context(
    contents=[
        t.DocumentContent(data=txt_url),
        t.TextContent(data="Summarize this document."),
    ]
)

# context = Context(
#     contents=[
#         t.DocumentTextContent(data=txt_url),
#         t.TextContent(data="Summarize this document."),
#     ]
# )

In [ ]:
test = await model.acreate(user_input=context)
test.display()